In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("netflix_titles.csv")

In [4]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
# creating another USEFUL  columns 
data['tags'] = data['director'] + ' ' + data['cast'] +' '+ data['listed_in'] +' '+data['description']

In [8]:
data = data[['show_id','type','title','tags']]

In [9]:
data

,show_id,type,title,tags
0,s1,Movie,Dick Johnson Is Dead,NaN
1,s2,TV Show,Blood & Water,NaN
2,s3,TV Show,Ganglands,"Julien Leclercq Sami Bouajila, Tracy Gotoas, S..."
3,s4,TV Show,Jailbirds New Orleans,NaN
4,s5,TV Show,Kota Factory,NaN
...,...,...,...,...
8802,s8803,Movie,Zodiac,"David Fincher Mark Ruffalo, Jake Gyllenhaal, R..."
8803,s8804,TV Show,Zombie Dumb,NaN
8804,s8805,Movie,Zombieland,"Ruben Fleischer Jesse Eisenberg, Woody Harrels..."
8805,s8806,Movie,Zoom,"Peter Hewitt Tim Allen, Courteney Cox, Chevy C..."


In [11]:
data.dropna(inplace = True)

/var/folders/5z/yyfxq7351wqf4j26kgytyg0r0000gn/T/ipykernel_18188/339833404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace = True)


In [12]:
data.shape

(5700, 4)

In [13]:
data.reset_index(inplace= True, drop = True)

In [14]:
data.shape

(5700, 4)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
# Text --> Numerical

In [19]:
cv = CountVectorizer(max_features = 5700, stop_words = 'english')
# creats shape of 5700 ROWS. For each row. A vector

In [23]:
# call algorithm, convert to UTF -8
vector = cv.fit_transform(data['tags'].values.astype('U')).toarray()
from sklearn.metrics.pairwise import cosine_similarity
# cosine will draw dimension in 5700, find difference in rcords and angle theta betweent them and provides similarity.

In [25]:
similarity = cosine_similarity(vector)

In [26]:
similarity

array([[1.        , 0.21764288, 0.05647825, ..., 0.        , 0.03121953,
        0.06024752],
       [0.21764288, 1.        , 0.02594996, ..., 0.02923527, 0.05737753,
        0.02768183],
       [0.05647825, 0.02594996, 1.        , ..., 0.03413944, 0.13400504,
        0.09697623],
       ...,
       [0.        , 0.02923527, 0.03413944, ..., 1.        , 0.1132277 ,
        0.03641785],
       [0.03121953, 0.05737753, 0.13400504, ..., 0.1132277 , 1.        ,
        0.07147417],
       [0.06024752, 0.02768183, 0.09697623, ..., 0.03641785, 0.07147417,
        1.        ]])

In [27]:
# similarity for a particular movie. 
data[data['title'] =='Zodiac'].index[0]

5696

In [28]:
similarity[5696]

array([0.02564946, 0.02357023, 0.05504819, ..., 0.06201737, 0.06085806,
       0.05872202])

In [31]:
#convert to list and enumerate, sort in reverse. we need data which is similar to that movie.
#values near to 1 for high similarity. Working with index
similar_movies =sorted(list(enumerate(similarity[5696])), reverse = True, key = lambda vector:vector[1])

In [34]:
for i in similar_movies[0:6]:
    print(data.iloc[i[0]].title)
    print(i[1])

Zodiac
1.0000000000000004
To and From New York
0.2637521893583148
Gothika
0.2637521893583148
WHAT DID JACK DO?
0.2449489742783178
Interrogation
0.223606797749979
The Highwaymen
0.2201927530252721


In [47]:
def recommend_movie(movie_name):
    movie_index = data[data['title'] == movie_name].index
    if len(movie_index) == 0:
        print("Movie not found in the dataset.")
        return
    movie_index = movie_index[0]
    similar_movies = sorted(list(enumerate(similarity[movie_index])), reverse=True, key=lambda x: x[1])
    for i in similar_movies[1:6]:
        print(data.iloc[i[0]]['title'] + 'with similarity of '+ str(round(i[1]*100,2)))
        


In [48]:
recommend_movie('Ganglands')

Bangkok Breakingwith similarity of 51.21
Dealerwith similarity of 50.83
The Forest of Love: Deep Cutwith similarity of 50.74
Sleepless Society: Two Pillows & A Lost Soulwith similarity of 49.67
Unlucky Ploywith similarity of 47.36


# Export

In [51]:
import pickle as pk

In [52]:
pk.dump(data,open('movie_dataset.pkl','wb'))
#pkl is extension of model, wb - with write access

In [53]:
# export similarity matrix
pk.dump(similarity, open('similarity.pkl','wb'))

In [ ]:
# With help of these 2 pickle files. create web appli